# Min-Max Normalization 적용한 lightfm model 학습

In [1]:
import os
import pandas as pd
import numpy as np
from time import time
from tqdm.notebook import tqdm
import pickle

from scipy.sparse import coo_matrix, csr_matrix

from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k

from hyperopt import fmin, hp, tpe, Trials

C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


min_max_item_features.csv
min_max_user_features.csv
rating.csv
robust_item_features.csv
robust_user_features.csv
whisky.csv

In [2]:
item_features = pd.read_csv("dataset/min_max_item_features.csv", index_col=0, encoding="UTF-8")
user_features = pd.read_csv("dataset/min_max_user_features.csv", index_col=0, encoding="UTF-8")
rating = pd.read_csv("dataset/rating.csv", index_col=0, encoding="UTF-8")
whisky = pd.read_csv("dataset/whisky.csv", index_col=0, encoding="UTF-8")

In [3]:
item_features = csr_matrix(item_features)
user_features = csr_matrix(user_features)

In [4]:
print(user_features)

  (0, 0)	5.0
  (0, 1)	0.3
  (0, 2)	0.85
  (0, 3)	0.5
  (0, 4)	0.3
  (0, 5)	0.2
  (0, 6)	0.8
  (0, 7)	0.8
  (0, 8)	0.85
  (0, 9)	0.1666666666666666
  (0, 10)	0.2
  (0, 11)	0.25
  (0, 12)	0.85
  (0, 13)	0.5263157894736842
  (1, 0)	5.0
  (1, 1)	0.3
  (1, 2)	0.3
  (1, 3)	0.6
  (1, 4)	0.4
  (1, 5)	0.7000000000000001
  (1, 6)	0.8
  (1, 7)	0.7000000000000001
  (1, 8)	0.7000000000000001
  (1, 9)	0.4444444444444445
  (1, 10)	0.6
  :	:
  (119512, 7)	0.4
  (119512, 8)	1.0
  (119512, 10)	0.55
  (119512, 12)	0.3
  (119512, 13)	0.1052631578947368
  (119513, 0)	1.0
  (119513, 1)	0.31
  (119513, 3)	0.4
  (119513, 4)	0.29
  (119513, 6)	0.3
  (119513, 7)	0.4
  (119513, 8)	1.0
  (119513, 10)	0.55
  (119513, 12)	0.3
  (119513, 13)	0.1052631578947368
  (119514, 0)	1.0
  (119514, 1)	0.31
  (119514, 3)	0.4
  (119514, 4)	0.29
  (119514, 6)	0.3
  (119514, 7)	0.4
  (119514, 8)	1.0
  (119514, 10)	0.55
  (119514, 12)	0.3
  (119514, 13)	0.1052631578947368


In [5]:
unique_user = rating["user_id"].unique()

In [6]:
unique_user

array([     0,      1,      2, ..., 119512, 119513, 119514], dtype=int64)

### make Interactions 

In [7]:
rating.shape

(908176, 3)

In [8]:
def create_user_item_interactions(rating, n_users, n_items):
    interactions = coo_matrix(
        (
            rating['rating'].values,
            (rating['user_id'].values, rating["whisky_id"].values)
        ),
        shape=(n_users, n_items)
    )
    return interactions

In [9]:
interactions = create_user_item_interactions(rating, rating["user_id"].nunique(), whisky["whisky_id"].nunique())

In [10]:
print(interactions)
interactions

  (0, 0)	9.0
  (1, 0)	7.0
  (2, 0)	9.0
  (3, 0)	10.0
  (4, 0)	9.0
  (5, 0)	7.0
  (6, 0)	10.0
  (7, 0)	7.0
  (8, 0)	10.0
  (9, 0)	10.0
  (10, 0)	9.0
  (11, 0)	9.0
  (12, 0)	8.0
  (13, 0)	8.0
  (14, 0)	10.0
  (15, 0)	10.0
  (16, 0)	9.0
  (17, 0)	9.0
  (18, 0)	8.0
  (19, 0)	9.0
  (20, 0)	9.0
  (21, 0)	10.0
  (22, 0)	9.0
  (23, 0)	9.0
  (24, 0)	9.0
  :	:
  (7636, 3531)	4.0
  (56292, 3531)	2.0
  (2670, 3531)	2.0
  (786, 3531)	2.0
  (5942, 3531)	6.0
  (15561, 3531)	2.0
  (25623, 3532)	6.0
  (2885, 3532)	6.0
  (7368, 3533)	2.0
  (69554, 3534)	10.0
  (4484, 3534)	2.0
  (85727, 3534)	5.0
  (43900, 3534)	4.0
  (16070, 3534)	8.0
  (29998, 3534)	8.0
  (7895, 3534)	4.0
  (104052, 3534)	5.0
  (31152, 3534)	2.0
  (119512, 3534)	6.0
  (2509, 3534)	4.0
  (4828, 3534)	4.0
  (119513, 3534)	2.0
  (119514, 3534)	8.0
  (3123, 3534)	4.0
  (95648, 3534)	2.0


<119515x3535 sparse matrix of type '<class 'numpy.float64'>'
	with 908176 stored elements in COOrdinate format>

## Train_Test data split

In [11]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

In [12]:
train_interactions, valid_interactions = random_train_test_split(train_interactions, test_percentage=0.2, random_state=42)

### Hyper Parameter Optimization by using HyperOPT

In [ ]:
trials = Trials()
space = [
    hp.choice('no_components', range(10,51,10)),
    hp.uniform('learning_rate', 0.001, 0.005,  0.01),
    hp.uniform('item_alpha', 1e-05, 5e-05),
    hp.uniform('user_alpha', 1e-05, 5e-05),
]

In [ ]:
def objective(params):
    no_components, learning_rate, item_alpha, user_alpha = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=0)

    model.fit(interactions=train_interactions,
              user_features=user_features,
              item_features=item_features,
              epochs=3,
              verbose=True)

    test_precision = precision_at_k(model, valid_interactions, k=10, item_features=item_features, user_features=user_features).mean()
    test_recall = recall_at_k(model, valid_interactions, k=10, item_features=item_features, user_features=user_features).mean()
    test_auc = auc_score(model, valid_interactions, item_features=item_features, user_features=user_features).mean()
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, auc_score: {:.5f}".format(
      no_components, learning_rate, item_alpha, test_precision, test_recall, test_auc))
    output = -test_auc


    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5, trials=trials)

best_params

In [ ]:
midel = LightFM(no_components=20,
                learning_rate=0.05,
                loss="warp",
                item_alpha=0.01,
                user_alpha=0.01,
                random_state=42
               )

In [ ]:
%time model.fit(interactions=train_interactions, user_features=user_features, item_features=item_features, epochs=10, verbose=True)

In [ ]:
model = LightFM(no_components=40,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=0.011370592645615374,
                item_alpha=1.975734039413079e-05,
                random_state=42)

In [ ]:
%time midel.fit(interactions=train_interactions, user_features=user_features, item_features=item_features, epochs=10, verbose=True)

In [ ]:
print("Train precision: %.5f" % precision_at_k(midel, test_interactions, user_features=user_features, item_features=item_features, k=9).mean())
print("Test precision: %.5f" % precision_at_k(midel, test_interactions,  user_features=user_features, item_features=item_features, k=9).mean())

In [ ]:
test_auc = auc_score(midel, test_interactions, user_features=user_features, item_features=item_features).mean()
print(test_auc)

## Optuna 사용한 HyperParameter 최적화
---
- learning_rate, alpha 등의 하이퍼 파라미터 값을 작게 설정했을 때보다 높게 설정했을 때 AUC Score가 높게 나오는 경향이 보였습니다.
- HyperOPT는 베이지안 최적화 접근 기반인데 이 부분에 제대로 알지 못하기 때문에 전체를 돌려보는 Optuna 방식으로 변경하겠습니다.
- Optuna의 경우 시각화도 가능하고, GridSearchCV보다 빠르다는 장점이 있습니다.

In [13]:
import optuna

In [14]:
def objective(trial):
    
    
    # 조정할 하이퍼 파라미터
    params = {
        "learning_schedule": 'adagrad',
        "loss": "warp",
        "random_state": 0,
        "no_components": trial.suggest_categorical("no_components", [10, 20, 30, 40, 50]),
        'learning_rate': trial.suggest_categorical("learning_rate", [0.0005, 0.001, 0.005,  0.01,  0.05]),
        'item_alpha': trial.suggest_categorical("item_alpha", [5e-04, 1e-03, 5e-03, 1e-02]),
        'user_alpha': trial.suggest_categorical("user_alpha", [5e-04, 1e-03, 5e-03, 1e-02]),
    }

    model = LightFM(**params)

    model.fit(interactions=train_interactions,
              user_features=user_features,
              item_features=item_features,
              epochs=3,
              verbose=True)

    test_precision = precision_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_recall = recall_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_auc = auc_score(model, valid_interactions, item_features=item_features, user_features=user_features).mean()
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, user_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, auc_score: {:.5f}".format(
      params["no_components"], params["learning_rate"], params["item_alpha"], params["user_alpha"], test_precision, test_recall, test_auc))
    return test_auc

In [15]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_trial.params)

[I 2023-03-13 20:39:04,459] A new study created in memory with name: no-name-512a595e-85c2-49aa-a0ff-f866e2b5004e
Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:09<00:00, 23.16s/it]
[I 2023-03-13 20:42:48,357] Trial 0 finished with value: 0.7446987628936768 and parameters: {'no_components': 20, 'learning_rate': 0.0005, 'item_alpha': 0.001, 'user_alpha': 0.005}. Best is trial 0 with value: 0.7446987628936768.


no_comp: 20, lrn_rate: 0.00050, item_alpha: 0.00100, user_alpha: 0.00500, precision: 0.00077, recall: 0.00268, auc_score: 0.74470


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [02:14<00:00, 44.95s/it]
[I 2023-03-13 20:49:13,855] Trial 1 finished with value: 0.7767640948295593 and parameters: {'no_components': 40, 'learning_rate': 0.005, 'item_alpha': 0.01, 'user_alpha': 0.01}. Best is trial 1 with value: 0.7767640948295593.


no_comp: 40, lrn_rate: 0.00500, item_alpha: 0.01000, user_alpha: 0.01000, precision: 0.00282, recall: 0.01199, auc_score: 0.77676


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:47<00:00, 35.84s/it]
[I 2023-03-13 20:54:24,165] Trial 2 finished with value: 0.745319664478302 and parameters: {'no_components': 30, 'learning_rate': 0.0005, 'item_alpha': 0.001, 'user_alpha': 0.001}. Best is trial 1 with value: 0.7767640948295593.


...

no_comp: 40, lrn_rate: 0.05000, item_alpha: 0.00500, user_alpha: 0.00050, precision: 0.00577, recall: 0.03327, auc_score: 0.80749


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:12<00:00, 24.33s/it]
[I 2023-03-14 01:53:30,949] Trial 48 finished with value: 0.7803113460540771 and parameters: {'no_components': 20, 'learning_rate': 0.01, 'item_alpha': 0.005, 'user_alpha': 0.0005}. Best is trial 3 with value: 0.8078765869140625.


no_comp: 20, lrn_rate: 0.01000, item_alpha: 0.00500, user_alpha: 0.00050, precision: 0.00307, recall: 0.01312, auc_score: 0.78031


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [02:17<00:00, 45.75s/it]
[I 2023-03-14 02:00:07,623] Trial 49 finished with value: 0.8045592308044434 and parameters: {'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.01}. Best is trial 3 with value: 0.8078765869140625.


no_comp: 40, lrn_rate: 0.05000, item_alpha: 0.01000, user_alpha: 0.01000, precision: 0.00567, recall: 0.03168, auc_score: 0.80456
{'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.001}


best : no_comp: 40, lrn_rate: 0.05000, item_alpha: 0.00500, user_alpha: 0.00050, precision: 0.00577, recall: 0.03327, auc_score: 0.80749

In [20]:
print("Best Params : {}".format(study.best_params))
print()
print("Best Trials : {}".format(study.best_trials))
print()
print("Best Values : {}".format(study.best_value))

Best Params : {'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.001}

Best Trials : [FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.8078765869140625], datetime_start=datetime.datetime(2023, 3, 13, 20, 54, 24, 166409), datetime_complete=datetime.datetime(2023, 3, 13, 21, 0, 42, 679591), params={'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.001}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'no_components': CategoricalDistribution(choices=(10, 20, 30, 40, 50)), 'learning_rate': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01, 0.05)), 'item_alpha': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01)), 'user_alpha': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01))}, trial_id=3, value=None)]

Best Values : 0.8078765869140625


### uniform distribution 기반으로 최적 파라미터를 찾습니다.
---
- 위 과정은 GridSearch 방식으로 직접 설정한 파라미터 값에서 최적 값을 찾았습니다.
- 그 결과를 바탕으로 적절한 범위를 설정해 좀 더 세세한 optimal hyper parameter value를 찾습니다.

In [22]:
def objective(trial):    
    # 조정할 하이퍼 파라미터
    params = {
        "learning_schedule": 'adagrad',
        "loss": "warp",
        "random_state": 0,
        "no_components": trial.suggest_int("no_components", 40, 100, 10),
        'learning_rate': trial.suggest_float("learning_rate", 0.005,  0.01),
        'item_alpha': trial.suggest_float("item_alpha", 1e-03, 1e-02),
        'user_alpha': trial.suggest_float("user_alpha", 1e-03, 1e-02),
    }

    model = LightFM(**params)

    model.fit(interactions=train_interactions,
              user_features=user_features,
              item_features=item_features,
              epochs=5,
              verbose=True)

    test_precision = precision_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_recall = recall_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_auc = auc_score(model, valid_interactions, item_features=item_features, user_features=user_features).mean()
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, user_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, auc_score: {:.5f}".format(
      params["no_components"], params["learning_rate"], params["item_alpha"], params["user_alpha"], test_precision, test_recall, test_auc))
    return test_auc

In [23]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_trial.params)

[I 2023-03-14 09:42:18,351] A new study created in memory with name: no-name-63b82246-2429-4b54-8b31-523a81431f02
Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [04:24<00:00, 52.86s/it]
[I 2023-03-14 09:51:18,814] Trial 0 finished with value: 0.7900952696800232 and parameters: {'no_components': 60, 'learning_rate': 0.008744355581243406, 'item_alpha': 0.0054353405577961655, 'user_alpha': 0.007493531192629571}. Best is trial 0 with value: 0.7900952696800232.


no_comp: 60, lrn_rate: 0.00874, item_alpha: 0.00544, user_alpha: 0.00749, precision: 0.00391, recall: 0.01776, auc_score: 0.79010


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [04:33<00:00, 54.76s/it]
[I 2023-03-14 10:00:15,041] Trial 1 finished with value: 0.7869731187820435 and parameters: {'no_components': 60, 'learning_rate': 0.00783405821538858, 'item_alpha': 0.005256311308722085, 'user_alpha': 0.0035617167199210715}. Best is trial 0 with value: 0.7900952696800232.


no_comp: 60, lrn_rate: 0.00783, item_alpha: 0.00526, user_alpha: 0.00356, precision: 0.00380, recall: 0.01700, auc_score: 0.78697


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [02:59<00:00, 35.95s/it]
[I 2023-03-14 10:06:21,205] Trial 2 finished with value: 0.789766788482666 and parameters: {'no_components': 40, 'learning_rate': 0.009113351699509237, 'item_alpha': 0.0029736887742244276, 'user_alpha': 0.004450398783566782}. Best is trial 0 with value: 0.7900952696800232.


...

no_comp: 100, lrn_rate: 0.00939, item_alpha: 0.00957, user_alpha: 0.00888, precision: 0.00386, recall: 0.01744, auc_score: 0.79423


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [08:17<00:00, 99.57s/it]
[I 2023-03-14 14:12:29,273] Trial 18 finished with value: 0.792930006980896 and parameters: {'no_components': 80, 'learning_rate': 0.00945874250102717, 'item_alpha': 0.008250309819740616, 'user_alpha': 0.007791842444526473}. Best is trial 11 with value: 0.7956605553627014.


no_comp: 80, lrn_rate: 0.00946, item_alpha: 0.00825, user_alpha: 0.00779, precision: 0.00385, recall: 0.01746, auc_score: 0.79293


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [07:04<00:00, 84.94s/it]
[W 2023-03-14 14:23:54,622] Trial 19 failed with parameters: {'no_components': 90, 'learning_rate': 0.008641169469838991, 'item_alpha': 0.0063916245049787405, 'user_alpha': 0.009686035732046409} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\SSAFY\AppData\Local\Temp\ipykernel_27684\134447992.py", line 22, in objective
    test_recall = recall_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\evaluation.py", line 148, in recall_at_k
    ranks = model.predict_rank(
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\l

KeyboardInterrupt: 

In [24]:
print("Best Params : {}".format(study.best_params))
print()
print("Best Trials : {}".format(study.best_trials))
print()
print("Best Values : {}".format(study.best_value))

Best Params : {'no_components': 100, 'learning_rate': 0.009940909725802523, 'item_alpha': 0.007755873955143551, 'user_alpha': 0.009853008299281606}

Best Trials : [FrozenTrial(number=11, state=TrialState.COMPLETE, values=[0.7956605553627014], datetime_start=datetime.datetime(2023, 3, 14, 11, 47, 1, 521114), datetime_complete=datetime.datetime(2023, 3, 14, 12, 6, 32, 43161), params={'no_components': 100, 'learning_rate': 0.009940909725802523, 'item_alpha': 0.007755873955143551, 'user_alpha': 0.009853008299281606}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'no_components': IntDistribution(high=100, log=False, low=40, step=10), 'learning_rate': FloatDistribution(high=0.01, log=False, low=0.005, step=None), 'item_alpha': FloatDistribution(high=0.01, log=False, low=0.001, step=None), 'user_alpha': FloatDistribution(high=0.01, log=False, low=0.001, step=None)}, trial_id=11, value=None)]

Best Values : 0.7956605553627014


no_comp: 100, lrn_rate: 0.00994, item_alpha: 0.00776, user_alpha: 0.00985, precision: 0.00417, recall: 0.01984, auc_score: 0.79566

Best Params : {'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.001}
Best Values : 0.8078765869140625

### Evaluation
---
- 1차 최적화 hyper parameter 결과
- Best Params : {'no_components': 40, 'learning_rate': 0.05, 'item_alpha': 0.01, 'user_alpha': 0.001}
- Best Values : 0.8078765869140625

In [36]:
min_max_lightfm = LightFM(
    no_components=40, learning_rate=0.05000, item_alpha=0.00500, user_alpha=0.00050,learning_schedule='adagrad',loss="warp", random_state=42
)
%time min_max_lightfm.fit(interactions=train_interactions, user_features=user_features, item_features=item_features, epochs=10, verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [05:30<00:00, 33.00s/it]

CPU times: total: 5min 25s
Wall time: 5min 30s


In [37]:
test_precision = precision_at_k(min_max_lightfm, test_interactions, user_features=user_features, item_features=item_features, k=9).mean()
test_recall = recall_at_k(min_max_lightfm, test_interactions,  user_features=user_features, item_features=item_features, k=9).mean()
test_auc = auc_score(min_max_lightfm, test_interactions, user_features=user_features, item_features=item_features).mean()

print("Train precision: %.5f" % test_precision)
print("Test precision: %.5f" % test_recall)
print("Test AUC Score : %.5f" % test_auc)

Train precision: 0.00731
Test precision: 0.04211
Test AUC Score : 0.81388


### save

In [38]:
# save the model to a file
with open('min_max_lightfm.pkl', 'wb') as f:
    pickle.dump(min_max_lightfm, f)